# AXI Stream Implementation of RSA

In [21]:
# Supporting code for implementing
import random

import math



def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a
# Function to generate public key
def pubKeyGen(p, q):
    #define phi with (p,q)
    phi = (p-1)*(q-1)
    # calculate/return a value for e (public key)
    e = 2
    while (e < phi):
        if(gcd(e, phi) == 1):
            break
        else:
            e=e+1
    return e

# Functions to generate public/private keys
def generate_keypair(prime1, prime2):

    n = prime1 * prime2
    phi = (prime1 - 1) * (prime2 - 1)

    # Generate a public key value e from the primes
    e = pubKeyGen(prime1, prime2)

    # Compute the private key d.
    d = pow(e, -1, phi)
 
    # Return the public and private keypairs
    return ((e, n), (d, n))

# Function to convert the message to an array of integers.
def prepInput(message):
    intArrString = [ord(char) for char in message]
    return intArrString
# Function to reverse the action above and return a string from an array of integers.
def convOutput(orgMsg):
    stringFrInt = [chr(char) for char in orgMsg]
    return ''.join(stringFrInt)

In [22]:
# Test of converting string to a list of int - Passed!
inStr = "Hello"
inStr = prepInput(inStr)
print(inStr)
inStr =convOutput(inStr)
print(inStr)

[72, 101, 108, 108, 111]
Hello


## Overlay Code

In [ ]:
# Implementation Framework for AXI Stream

from pynq import Overlay
# Bit file from Vivado
ol = Overlay('')

dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

hls_ip = ol.example_0 

CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81)

In [ ]:
from pynq import allocate
import numpy as np

# Adapted function to allocate the input and output buffers based on the amount of chars
def initilizeIO(message):

    input_buffer = allocate(shape=(len(message),), dtype=np.uint64)
    output_buffer = allocate(shape=(len(message),), dtype=np.uint64)
    return output_buffer, input_buffer



In [ ]:
# Example of using this code

# First establist the 